In [7]:
import uuid
import yaml
import time
from julep import Client

AGENT_UUID = uuid.uuid4()
MONITOR_SOCIAL_MEDIA_TASK_UUID = uuid.uuid4()
ANALYZE_TRENDS_TASK_UUID = uuid.uuid4()
SEND_ALERT_TASK_UUID = uuid.uuid4()

api_key = ""  # Your API key here
client = Client(api_key=api_key, environment="dev")

agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Social Media Monitor",
    about="An AI agent that monitors social media for keywords and trends.",
    model="gpt-4o",
)

monitor_social_media_task_def = yaml.safe_load("""
name: Monitor Social Media

input_schema:
  type: object
  properties:
    keywords:
      type: array
      items:
        type: string

main:
- prompt:
    - role: system
      content: >-
        Monitor the specified social media platforms for the following keywords:
        {{inputs[0].keywords}}.

        Return the posts containing these keywords.
  unwrap: true

- evaluate:
    posts: _

- return:
    posts: _
""")

monitor_social_media_task = client.tasks.create_or_update(
    task_id=MONITOR_SOCIAL_MEDIA_TASK_UUID,
    agent_id=AGENT_UUID,
    **monitor_social_media_task_def
)

analyze_trends_task_def = yaml.safe_load("""
name: Analyze Trends

input_schema:
  type: object
  properties:
    posts:
      type: array
      items:
        type: string
    threshold:
      type: integer

main:
- prompt:
    - role: system
      content: >-
        Analyze the following posts for trends and return any posts exceeding the threshold of
        {{inputs[0].threshold}} mentions:
        {{inputs[0].posts}}.

        Return the trending posts.
  unwrap: true

- evaluate:
    trends: _

- return:
    trends: _
""")

analyze_trends_task = client.tasks.create_or_update(
    task_id=ANALYZE_TRENDS_TASK_UUID,
    agent_id=AGENT_UUID,
    **analyze_trends_task_def
)

send_alert_task_def = yaml.safe_load("""
name: Send Alert

input_schema:
  type: object
  properties:
    trends:
      type: array
      items:
        type: string

main:
- prompt:
    - role: system
      content: >-
        Send an alert for the following trends detected on social media:
        {{inputs[0].trends}}.

        Confirm the alert has been sent.
  unwrap: true

- return:
    status: "Alert sent"
""")

send_alert_task = client.tasks.create_or_update(
    task_id=SEND_ALERT_TASK_UUID,
    agent_id=AGENT_UUID,
    **send_alert_task_def
)

def analyze_trends(posts, threshold):
    trends = [post for post in posts if posts.count(post) >= threshold]
    if trends:
        return {
            "trends": trends
        }
    else:
        return {"trends": []}

def send_alert(trends):
    if not trends:
        return {"status": "No trends detected to alert."}

    input_trends = {"trending_posts": trends}
    execution = client.executions.create(
        task_id=SEND_ALERT_TASK_UUID,
        input=input_trends
    )
    time.sleep(2)
    result = client.executions.get(execution.id)

    output = client.executions.transitions.list(execution_id=result.id).items[0].output
    return {"status": output}

if trends_result.get("trends"):
    extracted_trends = trends_result["trends"]
    alert_result = send_alert(extracted_trends)
else:
    alert_result = {"status": "No trends detected."}

print_output(monitor_result, trends_result, alert_result)


def print_output(monitor_result, trends_result, alert_result):
    print("Demonstrating Social Media Monitoring and Alert System:\n")

    print("Posts Monitored:")
    print(f"Posts containing specified keywords:\n{', '.join(monitor_result['posts'])}\n")

    print("Trends Analyzed:")
    print(f"Trending posts based on the defined threshold:\n{', '.join(trends_result['trends'])}\n")

    print("Alert Status:")
    print(f"Alert status: {alert_result.get('status', 'No alert sent')}\n")

keywords = ["AI", "Healthcare", "Data"]
threshold = 2

monitor_result = monitor_social_media(keywords)

monitor_result = {
    "posts": [
        "AI in Healthcare is revolutionizing patient care.",
        "Data-driven insights are essential for AI development.",
        "Healthcare providers are adopting AI technologies."
    ]
}

trends_result = analyze_trends(monitor_result["posts"], threshold)

print("Trends Result:", trends_result)

if trends_result.get("trends"):
    alert_result = send_alert(trends_result["trends"])
else:
    alert_result = {"status": "No trends detected."}

print_output(monitor_result, trends_result, alert_result)


Demonstrating Social Media Monitoring and Alert System:

Posts Monitored:
Posts containing specified keywords:
AI in Healthcare is revolutionizing patient care., Data-driven insights are essential for AI development., Healthcare providers are adopting AI technologies.

Trends Analyzed:
Trending posts based on the defined threshold:
trends

Alert Status:
Alert status: Please provide the details of the trends you would like to send an alert for, and I will assist you in confirming that the alert has been sent.

Trends Result: {'trends': []}
Demonstrating Social Media Monitoring and Alert System:

Posts Monitored:
Posts containing specified keywords:
AI in Healthcare is revolutionizing patient care., Data-driven insights are essential for AI development., Healthcare providers are adopting AI technologies.

Trends Analyzed:
Trending posts based on the defined threshold:


Alert Status:
Alert status: No trends detected.

